In [13]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool

ImportError: cannot import name 'ToolNode' from 'langgraph.prebuilt' (unknown location)

In [14]:
from typing import TypedDict
from typing_extensions import Literal
from langgraph.graph import END, StateGraph
from langgraph.types import Command


class InputState(TypedDict):
    string_value: str
    numeric_value: int


def modify_state(input_state: InputState) -> InputState:
    input_state["string_value"] += "a"
    input_state["numeric_value"] += 1
    return input_state


def branch_a(state: InputState) -> Command[Literal["branch_b"]]:
    print(f"branch_a: Current state: {state}")
    new_state = modify_state(state)
    print(f"branch_a: Updated state: {new_state}")
    return Command(
        goto="branch_b",
        update=new_state,
    )


def branch_b(state: InputState) -> Command[Literal["branch_c"]]:
    print(f"branch_b: Current state: {state}")
    new_state = modify_state(state)
    print(f"branch_b: Updated state: {new_state}")
    return Command(
        goto="branch_c",
        update=new_state,
    )


def branch_c(state: InputState) -> Command[Literal[END]]:
    print(f"branch_c: Current state: {state}")
    new_state = modify_state(state)
    print(f"branch_c: Updated state: {new_state}")
    return Command(goto=END, update=new_state)

In [15]:
graph = StateGraph(InputState)

graph.add_node("branch_a", branch_a)
graph.add_node("branch_b", branch_b)
graph.add_node("branch_c", branch_c)
graph.set_entry_point("branch_a")

runnable = graph.compile()

In [19]:
initial_state = {"string_value": "Hello", "numeric_value": 0}
final_state = runnable.invoke(initial_state)

print("Final state:", final_state)

Task branch_c with path ('__pregel_pull', 'branch_c') wrote to unknown channel branch:to:__end__, ignoring it.


branch_a: Current state: {'string_value': 'Hello', 'numeric_value': 0}
branch_a: Updated state: {'string_value': 'Helloa', 'numeric_value': 1}
branch_b: Current state: {'string_value': 'Helloa', 'numeric_value': 1}
branch_b: Updated state: {'string_value': 'Helloaa', 'numeric_value': 2}
branch_c: Current state: {'string_value': 'Helloaa', 'numeric_value': 2}
branch_c: Updated state: {'string_value': 'Helloaaa', 'numeric_value': 3}
Final state: {'string_value': 'Helloaaa', 'numeric_value': 3}


In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode, tools_condition

checkpointer = MemorySaver()

graph_builder = StateGraph(State)

ImportError: cannot import name 'ToolNode' from 'langgraph.prebuilt' (unknown location)

In [24]:
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["tools"],
)

NameError: name 'graph_builder' is not defined

In [29]:
from typing_extensions import Literal
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from IPython.display import Image, display
from langchain.agents import Tool
from langchain.agents.agent import AgentExecutor
from langgraph.prebuilt import ToolNode, tools_condition



@tool
def weather_search(city: str):
    """Search for the weather in a specific city"""
    print("----")
    print(f"Searching for: {city}")
    print("----")
    return "Sunny!"


model = ChatOpenAI(model="gpt-4o-mini").bind_tools([weather_search])


def call_llm(state):
    return {"messages": [model.invoke(state["messages"])]}


def human_review_node(state) -> Command[Literal["call_llm", "run_tool"]]:
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[-1]

    human_review = interrupt(
        {
            "question": "Is this correct?",
            "tool_call": tool_call,
        }
    )
    print("Human review values:", human_review)

    review_action = human_review["action"]
    review_data = human_review.get("data")

    if review_action == "continue":
        return Command(goto="run_tool")

    elif review_action == "update":
        updated_message = {
            "role": "ai",
            "content": last_message.content,
            "tool_calls": [
                {
                    "id": tool_call["id"],
                    "name": tool_call["name"],
                    "args": review_data,
                }
            ],
            "id": last_message.id,
        }
        return Command(goto="run_tool", update={"messages": [updated_message]})


def route_after_llm(state) -> Literal[END, "human_review_node"]:
    if len(state["messages"][-1].tool_calls) == 0:
        return END
    else:
        return "human_review_node"


builder = StateGraph(MessagesState)
builder.add_node("call_llm", call_llm)
builder.add_node("run_tool", ToolNode(tools=[weather_search]))
builder.add_node("human_review_node", human_review_node)
builder.add_conditional_edges("call_llm", route_after_llm)
builder.add_edge("run_tool", "call_llm")
builder.set_entry_point("call_llm")

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

ImportError: cannot import name 'ToolNode' from 'langgraph.prebuilt' (unknown location)

In [41]:
from langgraph.graph import StateGraph, START, END

# --- Define State ---
class State(dict):
    pass

# --- Mock Tool ---
def get_weather(city):
    weather_map = {
        "london": "Sunny, 22°C",
        "paris": "Cloudy, 18°C",
        "tokyo": "Rainy, 24°C"
    }
    return weather_map.get(city.lower(), "No data")

# --- LLM Node ---
def llm_node(state):
    messages = state.get("messages", [])
    if not messages:
        return {"messages": [{"role": "assistant", "content": "Ask me about the weather!"}], "pending_tool": None}

    last_msg = messages[-1]["content"].lower()
    if "weather" in last_msg:
        words = last_msg.split()
        city = "London"
        for i, word in enumerate(words):
            if word in ["in", "for"] and i+1 < len(words):
                city = words[i+1].title()
                break

        return {
            "messages": messages + [{"role": "assistant", "content": f"Should I fetch weather for {city}?"}],
            "pending_tool": {"name": "get_weather", "args": {"city": city}}
        }

    return {"messages": messages + [{"role": "assistant", "content": "Ask something like: 'What's the weather in Paris?'"}], "pending_tool": None}

# --- Human Approval ---
def human_approval(state):
    tool = state["pending_tool"]
    city = tool["args"]["city"]

    print(f"\n🔧 Tool request: get_weather for {city}")
    choice = input("Approve? (y/n/edit): ").strip().lower()

    if choice == "y":
        return state
    elif choice == "edit":
        new_city = input("Enter new city: ").strip()
        tool["args"]["city"] = new_city.title()
        return state
    else:
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": "Tool use canceled."}],
            "pending_tool": None
        }

# --- Tool Execution ---
def execute_tool(state):
    city = state["pending_tool"]["args"]["city"]
    result = get_weather(city)
    return {
        "messages": state["messages"] + [{"role": "assistant", "content": f"🌤️ Weather in {city}: {result}"}],
        "pending_tool": None
    }

# --- Routing ---
def decide_next(state):
    return "human_approval" if state.get("pending_tool") else END

def after_approval(state):
    return "execute_tool" if state.get("pending_tool") else END

# --- Build Workflow ---
builder = StateGraph(State)
builder.add_node("llm", llm_node)
builder.add_node("human_approval", human_approval)
builder.add_node("execute_tool", execute_tool)
builder.add_edge(START, "llm")
builder.add_conditional_edges("llm", decide_next)
builder.add_conditional_edges("human_approval", after_approval)
builder.add_edge("execute_tool", END)
app = builder.compile()

# --- CLI Chat ---
def chat():
    print("🌤️ Ask about the weather! Type 'quit' to exit.")
    state = {"messages": [], "pending_tool": None}

    while True:
        user_input = input("\n👤 You: ").strip()
        if user_input.lower() in ["quit", "exit"]:
            break
        state["messages"].append({"role": "user", "content": user_input})
        state = app.invoke(state)
        print("🤖 " + state["messages"][-1]["content"])

if __name__ == "__main__":
    chat()


🌤️ Ask about the weather! Type 'quit' to exit.


TypeError: 'NoneType' object is not subscriptable

In [30]:
import langgraph
print(langgraph.__version__)

AttributeError: module 'langgraph' has no attribute '__version__'

In [34]:
# Try these alternatives:
try:
    from langgraph.prebuilt import ToolNode, tools_condition
except ImportError:
    try:
        from langgraph.prebuilt.tool_node import ToolNode
        from langgraph.prebuilt.tool_executor import tools_condition
    except ImportError:
        try:
            from langgraph.prebuilt import create_tool_calling_executor
            # Use create_tool_calling_executor instead
        except ImportError:
            print("ToolNode not available in this version")

ToolNode not available in this version
